# Anomaly detection over heterogeneous data streams

The recipe, inspired from [this paper](https://arxiv.org/abs/1812.02848) by Palladino and Thissen:

1. Split time line into intervals; split the data streams into subsets corresponding to these intervals.
1. For each interval:
    1. Compose a graph out of the events belonging to the interval.
    1. Extract a vector topological features for each vertex.
    1. Reduce the dimension of these feature vectors.
    1. Compute the anomaly indicator according to the reduced feature vectors for the interval.
    
These various steps will be detailed as we go along.

In [1]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 120 --ignore W293,E302
%pdb on

Automatic pdb calling has been turned ON


In [2]:
import notebooks_as_modules

In [3]:
from jupytest import Suite, Report, Magic, fail, summarize_results, assert_, eq, Explanation, ExplanationOnFailure, join_args

In [4]:
suite = Suite('test')
if __name__ == '__main__':
    suite |= Report()
    suite |= Magic()

In [5]:
from growing import growing

## Time line partition

Each data stream is taken in as a Dask dataframe, itself indexed by a certain key. A heterogeneous data stream is a set of named data streams, all indexed in the same key.

In [6]:
import dask.dataframe as ddf
from typing import Mapping, TypeVar

DomainIndex = TypeVar("DomainIndex")
Schema = Mapping[str, str]
StreamHeterogeneous = Mapping[str, ddf.DataFrame]  # Data frame is indexed by DomainIndex

Given a subset of index values (articulated as an ordered, strictly monotonic increasing iterable sequence), a *partition* is thus a heterogeneous stream, mapped by these values.

In [7]:
import pandas as pd
from typing import Mapping, Tuple

Partition = Tuple[DomainIndex, StreamHeterogeneous]
Partitioning = Mapping[DomainIndex, StreamHeterogeneous]

### Partitioning sequence

The partitioning sequence is processed into a sequence of slices, with which to interrogate the index of the heterogeneous stream. Namely, a sequence `[i0, i1, i2... iN]` yields mutually exclusive slices `i0:(i1-1)`, `i1:(i2-1)`... `iN-1:((iN)-1)`. Let's embody partition sequences into a class.

In [8]:
from abc import ABC, abstractmethod
from typing import Generic, Sequence, Iterator, Tuple


class SequencePartition(ABC, Generic[DomainIndex]):

    @abstractmethod
    def __iter__(self) -> Iterator[DomainIndex]:
        raise NotImplementedError()
        yield 0
        
    @abstractmethod
    def pred(self, n: DomainIndex) -> DomainIndex:
        raise NotImplementedError()
        
    def slices(self) -> Iterator[Tuple[DomainIndex, DomainIndex]]:
        prev: Optional[DomainIndex] = None
        for cur in self:
            if prev is not None:
                yield (prev, self.pred(cur))
            prev = cur

In [9]:
class SequenceInt(SequencePartition[int]):
    
    def __init__(self, seq: Sequence[int]) -> None:
        super().__init__()
        self._seq = seq
        
    def __iter__(self) -> Iterator[int]:
        return iter(self._seq)
    
    def pred(self, n: int) -> int:
        return n - 1

In [10]:
%%test No slice for a sequence of less than two items
for i, seq in enumerate([[], [0]]):
    assert_(eq, actual=len(list(SequenceInt(seq).slices())), expected=0)

Test No slice for a sequence of less than two items passed.


In [11]:
%%test Iterating single slice
assert_(eq, expected=[(0, 1)], actual=list(SequenceInt([0, 2]).slices()))

Test Iterating single slice passed.


In [12]:
%%test With sequences not strictly monotonic, we get degenerate slices
assert_(eq, expected=[(0, 0), (1, 0)], actual=list(SequenceInt([0, 1, 1]).slices()))

Test With sequences not strictly monotonic, we get degenerate slices passed.


In [13]:
%%test Multiple slices from a sequence of integers
assert_(eq, expected=[(0, 2), (3, 4)], actual=list(SequenceInt([0, 3, 5]).slices()))

Test Multiple slices from a sequence of integers passed.


### Partitioning sequence of date-times

In [14]:
from typing import Any


class SequenceDatetime(SequencePartition[pd.Timestamp]):
    
    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__()
        self._seq = pd.date_range(*args, **kwargs)
        
    def __iter__(self) -> Iterator[pd.Timestamp]:
        return iter(self._seq)
        
    def pred(self, ts: pd.Timestamp) -> pd.Timestamp:
        return ts - pd.Timedelta(nanoseconds=1)

In [15]:
%%test Multiple slices from a date-time partitioning sequence
assert_(
    eq,
    expected=[
        (pd.Timestamp("2015-01-01T00:00:00"), pd.Timestamp("2015-01-01T00:59:59.999999999")),
        (pd.Timestamp("2015-01-01T01:00:00"), pd.Timestamp("2015-01-01T01:59:59.999999999")),
        (pd.Timestamp("2015-01-01T02:00:00"), pd.Timestamp("2015-01-01T02:59:59.999999999")),
        (pd.Timestamp("2015-01-01T03:00:00"), pd.Timestamp("2015-01-01T03:59:59.999999999"))
    ],
    actual=list(SequenceDatetime("2015-01-01T00:00:00", "2015-01-01T04:00:00", freq=pd.Timedelta(hours=1)).slices())
)

Test Multiple slices from a date-time partitioning sequence passed.


### Partitioning of the heterogeneous stream

In [16]:
def partition(sh: StreamHeterogeneous, seq: SequencePartition) -> Partitioning:
    P = {}
    for lower, upper in seq.slices():
        P[lower] = {name: df.loc[lower:upper] for name, df in sh.items()}
    return P

In [17]:
import dask.dataframe as ddf
from jupytest import Explanation
import pandas as pd


def equals(expected: pd.DataFrame, actual: ddf.DataFrame) -> Explanation:
    if actual.compute().equals(expected):
        return True
    return Explanation(
        "These dataframes are not equal as expected",
        [("Expected", expected), ("Actual", actual)]
    )

In [18]:
%%test Partitioning an int-indexed heterogeneous stream
import dask


@dask.delayed
def f1():
    return pd.DataFrame({"a": [3, 8, 9], "b": [10, 102, 89]}, index=pd.Int64Index([1, 4, 6]))

@dask.delayed
def f2():
    return pd.DataFrame({"a": [5, 3], "b": [90, 32]}, index=pd.Int64Index([8, 11]))

@dask.delayed
def f3():
    return pd.DataFrame({"a": [2, 11, 7, 3], "b": [56, 98, 99, 67]}, index=pd.Int64Index([13, 14, 15, 19]))

@dask.delayed
def g1():
    return pd.DataFrame(
        {"a": [6, 3, 8, -1, 0], "c": [-3, -8, -11, -7, -10], "d": [106, 203, 267, 308, 429]},
        index=pd.Int64Index([3, 6, 7, 10, 11])
    )

@dask.delayed
def g2():
    return pd.DataFrame(
        {"a": [2, 8, 9, 5, 5, 11], "c": [-13, -6, -7, -6, -8, -2], "d": [567, 308, 289, 290, 432, 387]},
        index=pd.Int64Index([14, 15, 17, 19, 21, 24])
    )


sh = {
    "p": ddf.from_delayed(
        [f1(), f2(), f3()],
        meta={"a": "int64", "b": "int64"},
        divisions=[1, 8, 13, 20]
    ),
    "q": ddf.from_delayed(
        [g1(), g2()],
        meta={"a": "int64", "c": "int64", "d": "int64"}, 
        divisions=[3, 14, 28]
    )
}
P = partition(sh, SequenceInt([0, 5, 10, 15, 20]))

assert_(eq, actual=set(P.keys()), expected={0, 5, 10, 15})
assert_(eq, reference={"p", "q"}, **{str(i): set(p.keys()) for i, p in enumerate(P.values())})

assert_(equals, actual=P[0]["p"], expected=pd.DataFrame({"a": [3, 8], "b": [10, 102]}, index=[1, 4]))
assert_(equals, actual=P[0]["q"], expected=pd.DataFrame({"a": [6], "c": [-3], "d": [106]}, index=[3]))
assert_(equals, actual=P[5]["p"], expected=pd.DataFrame({"a": [9, 5], "b": [89, 90]}, index=[6, 8]))
assert_(
    equals,
    actual=P[5]["q"],
    expected=pd.DataFrame({"a": [3, 8], "c": [-8, -11], "d": [203, 267]}, index=[6, 7])
)
assert_(
    equals,
    actual=P[10]["p"],
    expected=pd.DataFrame({"a": [3, 2, 11], "b": [32, 56, 98]}, index=[11, 13, 14])
)
assert_(
    equals,
    actual=P[10]["q"],
    expected=pd.DataFrame(
        {"a": [-1, 0, 2], "c": [-7, -10, -13], "d": [308, 429, 567]},
        index=[10, 11, 14]
    )
)
assert_(
    equals,
    actual=P[15]["p"],
    expected=pd.DataFrame({"a": [7, 3], "b": [99, 67]}, index=[15, 19])
)
assert_(
    equals,
    actual=P[15]["q"],
    expected=pd.DataFrame(
        {"a": [8, 9, 5], "c": [-6, -7, -6], "d": [308, 289, 290]},
        index=[15, 17, 19]
    )
)

Test Partitioning an int-indexed heterogeneous stream passed.


## Crunching event sets into artifact graphs

Step 2A above creating an *artifact graph* for each data partition. These graphs involve the decomposition of each event of a partition into a set of *artifacts*, entities associated to typical IT concepts, often acting as signifiers for a host, a computation, a network. The artifact graph is composed as follows:

1. Each artifact is associated to a vertex in the graph.
1. The artifacts extracted from an event E form a clique within the graph.
1. When adding an event clique to the graph, if an edge already exist, we increment its weight.

### Graph architecture

Determining (and engineering) which elements of an event are artifacts is a matter of arbitrary policy. We will encode this policy into a *graph architecture* object.

In [19]:
from abc import ABC, abstractmethod
from typing import Mapping, Sequence, NewType, NamedTuple


Artifact = Tuple[str, str]
Schema = Mapping[str, str]


class ArchGraph(ABC):
    
    @abstractmethod
    def get_artifacts(self, event: NamedTuple, schema: Schema) -> Sequence[Artifact]:
        """
        Extracts the artifacts from an event, itself encoded as a ad hoc namedtuple (hence the absence)
        of typing information.
        """
        raise NotImplementedError()

The typical architecture is to use all event attributes of dtype `object` as artifacts.

In [20]:
class AllObjects(ArchGraph):
    
    def get_artifacts(self, event: NamedTuple, dtypes: Schema) -> Sequence[Artifact]:
        artifacts = []
        for attr, dtype in dtypes.items():
            if dtype == "object":
                try:
                    artifacts.append((attr, getattr(event, attr)))
                except AttributeError:
                    # We ignore the absence of any attribute.
                    pass
        return artifacts

An event factory for testing:

In [21]:
from collections import namedtuple
from contextlib import contextmanager
from typing import ContextManager


@contextmanager
def test_event_factory(**fields: Any) -> ContextManager[namedtuple]:
    type_event = namedtuple("TestEvent", fields.keys())
    yield type_event(**fields)

In [22]:
%%test No artifact provided when the dtype catalog is empty
with test_event_factory(a="asdf", b="qwer", c=38) as event:
    assert AllObjects().get_artifacts(event, {}) == []

Test No artifact provided when the dtype catalog is empty passed.


In [23]:
%%test Event with no attribute yields no artifact
with test_event_factory() as event:
    assert AllObjects().get_artifacts(event, {"a": "object", "b": "object", "c": "int64"}) == []

Test Event with no attribute yields no artifact passed.


In [24]:
%%test AllObjects yields all object attributes as artifacts
with test_event_factory(a="asdf", b="qwer", c=38) as event:
    assert sorted(AllObjects().get_artifacts(event, {"a": "object", "b": "object", "c": "int64"})) == \
        [("a", "asdf"), ("b", "qwer")]

Test AllObjects yields all object attributes as artifacts passed.


In [25]:
%%test Missing artifact compared to the dtype catalog is silently ignored
with test_event_factory(a="asdf", c=38) as event:
    assert AllObjects().get_artifacts(event, {"a": "object", "b": "object", "c": "int64"}) == [("a", "asdf")]

Test Missing artifact compared to the dtype catalog is silently ignored passed.


In [26]:
%%test Extraneous attributes compared to the dtype catalog are ignored
with test_event_factory(a="asdf", b="qwer", d="zxcv") as event:
    assert sorted(AllObjects().get_artifacts(event, {"a": "object", "b": "object", "c": "int64"})) == \
        [("a", "asdf"), ("b", "qwer")]

Test Extraneous attributes compared to the dtype catalog are ignored passed.


### Adding artifact cliques to a graph

In [27]:
import igraph as ig
from itertools import combinations


def add_clique(g: ig.Graph, clique: Sequence[Artifact]) -> ig.Graph:
    dummy = g.add_vertex("__DUMMY__")  # Necessary to query the main vertex sequence.

    vertices = []
    for type_artifact, value_artifact in clique:
        name = f"{type_artifact}:{value_artifact}"
        vertices += (g.vs(name=name) or [g.add_vertex(name)])

    for src, dst in combinations(vertices, 2):
        for edge in (g.es(_from=src, _to=dst) or [g.add_edge(src, dst, weight=0)]):
            edge["weight"] += 1
            
    g.delete_vertices(dummy)
    return g

In [28]:
import igraph as ig


def test_graph() -> ig.Graph:
    g = ig.Graph()
    v_asdf = g.add_vertex("a:asdf")
    v_qwer = g.add_vertex("b:qwer")
    g.add_edge(v_asdf, v_qwer, weight=1)
    return g

#### Comparing graphs by reducing them to tuples and sets

In [29]:
from typing import List


ListVertices = List[str]
ListEdgesWeighted = List[Tuple[str, str, int]]
GraphLists = Tuple[ListVertices, ListEdgesWeighted]


def graph2sets(g: ig.Graph) -> GraphLists:
    return (
        sorted([v["name"] for v in g.vs]),
        sorted([tuple(sorted([v["name"] for v in e.vertex_tuple])) + (e["weight"],) for e in g.es])
    )

In [30]:
%%test Graph sets for simple graph
assert_(eq, actual=graph2sets(test_graph()), expected=(["a:asdf", "b:qwer"], [("a:asdf", "b:qwer", 1)]))

Test Graph sets for simple graph passed.


In [31]:
%%test Graph sets for bit more complex graph
g = ig.Graph()
for name in ["a", "b", "c", "1", "2", "3"]:
    g.add_vertex(name)
g.add_edge("a", "b", weight=1)
g.add_edge("c", "b", weight=3)
g.add_edge("1", "2", weight=5)
g.add_edge("3", "2", weight=0)
g.add_edge("b", "2", weight=10)

assert_(
    eq,
    expected=(
        ["1", "2", "3", "a", "b", "c"],
        [("1", "2", 5), ("2", "3", 0), ("2", "b", 10), ("a", "b", 1), ("b", "c", 3)]
    ),
    actual=graph2sets(g)
)

Test Graph sets for bit more complex graph passed.


In [32]:
%%test Graph sets don't care about vertex addition order
g1 = ig.Graph()
g1.add_vertex("a")
g1.add_vertex("b")
g1.add_edge("a", "b", weight=1)

g2 = ig.Graph()
g2.add_vertex("b")
g2.add_vertex("a")
g2.add_edge("b", "a", weight=1)

assert_(eq, ordered=graph2sets(g1), unordered=graph2sets(g2))

Test Graph sets don't care about vertex addition order passed.


In [33]:
%%test Graph sets when no edge
g = ig.Graph()
for name in ["a", "b"]:
    g.add_vertex(name)
    
assert_(eq, actual=graph2sets(g), expected=(["a", "b"], []))

Test Graph sets when no edge passed.


#### Unit tests for clique adding

In [34]:
import igraph as ig

def are_graphs_equal(**graphs: ig.Graph) -> ExplanationOnFailure:
    sets = {name: graph2sets(g) for name, g in graphs.items()}
    if eq(**sets):
        return True
    return Explanation("These graphs are not equal", join_args([], sets))

In [35]:
%%test Adding a clique without any artifact leaves the graph unchanged
expected = test_graph()
actual = add_clique(test_graph(), [])
assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique without any artifact leaves the graph unchanged passed.


In [36]:
%%test Add a clique to an empty graph
expected = test_graph()
actual = ig.Graph()
assert_(are_graphs_equal, expected=expected, actual=add_clique(ig.Graph(), [("a", "asdf"), ("b", "qwer")]))

Test Add a clique to an empty graph passed.


In [37]:
%%test Adding a clique of all new vertices
clique: Sequence[Artifact] = [("c", "uiop"), ("d", "qwer"), ("e", "zxcv"), ("f", "hjkl")]

expected = test_graph()
for t, v in clique:
    expected.add_vertex(f"{t}:{v}")
for s, e in [
    ("c:uiop", "d:qwer"), ("c:uiop", "e:zxcv"), ("c:uiop", "f:hjkl"),
    ("d:qwer", "e:zxcv"), ("d:qwer", "f:hjkl"),
    ("e:zxcv", "f:hjkl")
]:
    expected.add_edge(s, e, weight=1)

actual = add_clique(test_graph(), clique)
assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique of all new vertices passed.


In [38]:
%%test Adding a clique that intersects another one fully
clique: Sequence[Artifact] = [("a", "asdf"), ("b", "qwer"), ("c", "zxcv")]

expected = test_graph()
a = next(iter(expected.vs(name="a:asdf")))
b = next(iter(expected.vs(name="b:qwer")))
c = expected.add_vertex("c:zxcv")
for e in expected.es(_from=a, _to=b):
    e["weight"] = 2
expected.add_edge(a, c, weight=1)
expected.add_edge(b, c, weight=1)

actual = add_clique(test_graph(), clique)
assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique that intersects another one fully passed.


In [39]:
%%test Adding a clique that intersects another one by one vertex only
clique = [("b", "qwer"), ("c", "zxcv")]

expected = test_graph()
b = next(iter(expected.vs(name="b:qwer")))
c = expected.add_vertex("c:zxcv")
expected.add_edge(b, c, weight=1)

actual = add_clique(test_graph(), clique)
assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique that intersects another one by one vertex only passed.


In [40]:
%%test Adding a clique that intersects another partly, but by more than a single vertex
clique = [("a", "asdf"), ("c", "zxcv"), ("d", "hjkl")]

expected = test_graph()
a = next(iter(expected.vs(name="a:asdf")))
b = next(iter(expected.vs(name="b:qwer")))
c = expected.add_vertex("c:zxcv")
expected.add_edge(a, c, weight=2)
expected.add_edge(b, c, weight=1)
d = expected.add_vertex("d:hjkl")
expected.add_edge(a, d, weight=1)
expected.add_edge(c, d, weight=1)

actual = test_graph()
actual.add_vertex("c:zxcv")
actual.add_edge(a, c, weight=1)
actual.add_edge(b, c, weight=1)
add_clique(actual, clique)

assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique that intersects another partly, but by more than a single vertex passed.


### Building the graph

In [41]:
import igraph as ig
from memo import memo


@memo
def build_graph_artifacts(sh: StreamHeterogeneous, arch: ArchGraph) -> ig.Graph:
    graph = ig.Graph()
    for name_stream, df_stream in sh.items():
        schema: Schema = {n: str(t) for n, t in df_stream.dtypes.items()}
        for event in df_stream.itertuples():
            add_clique(graph, arch.get_artifacts(event, schema))
    return graph

In [42]:
%%test Building an artifact graph
from memo import suspending_memoization

sh = {
    "asdf": pd.DataFrame(
        data={
            "X": ["a.b.a.b", "a.c.c.b", "a.b.a.c", "a.a.a.a", "b.a.b.c"],
            "Y": ["443", "21", "22", "443", "443"]
        },
        index=[1, 3, 7, 8, 9]
    ),
    "qwer": pd.DataFrame(
        data={
            "W": ["10.2.3.1", "10.2.128.3", "10.2.1.1", "10.10.1.1", "10.2.128.38", "10.10.1.43"],
            "X": ["a.b.a.b", "a.e.a.d", "a.a.b.b", "a.b.a.c", "b.c.b.c", "a.a.a.a"],
            "Z": ["asdf.exe", "qwer.exe", "zxcv.exe", "qwer.exe", "./hkjl", "asdf.exe"]
        },
        index=[0, 3, 4, 6, 8, 9]
    )
}

expected = ig.Graph()
for v in [
    "X:a.b.a.b", "X:a.c.c.b", "X:a.b.a.c", "X:a.a.a.a", "X:b.a.b.c", "X:a.e.a.d", "X:a.a.b.b", "X:b.c.b.c",
    "Y:443", "Y:21", "Y:22",
    "W:10.2.3.1", "W:10.2.128.3", "W:10.2.1.1", "W:10.10.1.1", "W:10.2.128.38", "W:10.10.1.43",
    "Z:asdf.exe", "Z:qwer.exe", "Z:zxcv.exe", "Z:./hkjl"
]:
    expected.add_vertex(v)
for s, e in zip(
    ["X:a.b.a.b", "X:a.c.c.b", "X:a.b.a.c", "X:a.a.a.a", "X:b.a.b.c"],
    ["Y:443", "Y:21", "Y:22", "Y:443", "Y:443"]
):
    expected.add_edge(s, e, weight=1)
for a, b, c in zip(
    ["W:10.2.3.1", "W:10.2.128.3", "W:10.2.1.1", "W:10.10.1.1", "W:10.2.128.38", "W:10.10.1.43"],
    ["X:a.b.a.b", "X:a.e.a.d", "X:a.a.b.b", "X:a.b.a.c", "X:b.c.b.c", "X:a.a.a.a"],
    ["Z:asdf.exe", "Z:qwer.exe", "Z:zxcv.exe", "Z:qwer.exe", "Z:./hkjl", "Z:asdf.exe"]
):
    expected.add_edge(a, b, weight=1)
    expected.add_edge(a, c, weight=1)
    expected.add_edge(b, c, weight=1)

with suspending_memoization():
    assert_(are_graphs_equal, expected=expected, actual=build_graph_artifacts(sh, AllObjects()))

Test Building an artifact graph passed.


# Test result summary

In [43]:
if __name__ == "__main__":
    _ = summarize_results(suite)

22 passed, 0 failed, 0 raised an error
